# Методы для работы с текстом
Для стандартных строк Python доступно много методов для обработки текста

In [2]:
my_string = 'hello'

In [4]:
my_string.capitalize()

'Hello'

In [6]:
my_string.isdigit()

False

In [8]:
help(str)

Help on class str in module builtins:

class str(object)
 |  str(object='') -> str
 |  str(bytes_or_buffer[, encoding[, errors]]) -> str
 |  
 |  Create a new string object from the given object. If encoding or
 |  errors is specified, then the object must expose a data buffer
 |  that will be decoded using the given encoding and error handler.
 |  Otherwise, returns the result of object.__str__() (if defined)
 |  or repr(object).
 |  encoding defaults to sys.getdefaultencoding().
 |  errors defaults to 'strict'.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __format__(self, format_spec, /)
 |      Return a formatted version of the string as described by format_spec.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  

# Pandas и работа с текстом
Pandas может сделать с текстом намного больше Полная документация по таким методам строк как индексация и регулярные выражения:
https://pandas.pydata.org/docs/user_guide/text.html

## Методы Pandas для работы с текстом в строковой (текстовой) колонке

In [12]:
import numpy as np
import pandas as pd

In [16]:
email = 'first@mail.com'

In [18]:
email.split('@')

['first', 'mail.com']

In [20]:
names = pd.Series(['andrew', 'bob', 'claire', 'david', '5'])
names

0    andrew
1       bob
2    claire
3     david
4         5
dtype: object

In [22]:
names.str.upper()

0    ANDREW
1       BOB
2    CLAIRE
3     DAVID
4         5
dtype: object

In [24]:
# Изменения в исходный объект не внеслись
names

0    andrew
1       bob
2    claire
3     david
4         5
dtype: object

In [26]:
names.str.isdigit()

0    False
1    False
2    False
3    False
4     True
dtype: bool

In [28]:
names.str.isalpha()

0     True
1     True
2     True
3     True
4    False
dtype: bool

## Разбиение строки на части, извлечение отдельных частей из строки (string)

In [31]:
# Текстовые значения разделённые запятыми, но в виде одной строки
tech_finance = ['GOOG,APPL,AMZN', 'JPN,BAC,GS']

In [33]:
len(tech_finance)

2

In [35]:
# Создадим объект Series
tickers = pd.Series(tech_finance)
tickers

0    GOOG,APPL,AMZN
1        JPN,BAC,GS
dtype: object

In [39]:
tickers.str.split(',')

0    [GOOG, APPL, AMZN]
1        [JPN, BAC, GS]
dtype: object

In [41]:
tech = 'GOOG,APPL,AMZN'

In [43]:
tech.split(',')[0]

'GOOG'

In [53]:
# В таком случае мы получим первые элементы для каждого из списков
tickers.str.split(',').str[0]

0    GOOG
1     JPN
dtype: object

In [55]:
tickers.str.split(',').str[0][0]

'GOOG'

In [65]:
tickers.str.split(',').str[0][1]

'JPN'

In [67]:
tickers.str.split(',').str[2][1]

'GS'

In [71]:
# Получим датафрейм
tickers.str.split(',', expand=True)

,0,1,2
0,GOOG,APPL,AMZN
1,JPN,BAC,GS


## Очистка и редактирование строк
### Применение методов один за одним

In [78]:
# Обратите внимание на "сдвиги строк" из-за дополнителных пробелов 
# в значениях "andrew  " и "  claire  "
messy_names = pd.Series(['andrew ', 'bo;b', '  claire  '])
messy_names

0       andrew 
1          bo;b
2      claire  
dtype: object

In [80]:
messy_names[0]

'andrew '

In [88]:
# Для начала уберём ;, затем уберём пробелы слева и справа от слов и выполним капитализацию слов
messy_names.str.replace(';', '').str.strip().str.capitalize()

0    Andrew
1       Bob
2    Claire
dtype: object

## Альтернатива - использование метода apply() со своей собственной функцией

In [91]:
def cleanup(name):
    """Функция для очистки и исправления записи имён"""
    name = name.replace(';', '')
    name = name.strip()
    name = name.capitalize()
    return name

In [93]:
messy_names.apply(cleanup)

0    Andrew
1       Bob
2    Claire
dtype: object

## Какой вариант будет более эффективным?

In [96]:
import timeit

# Этот фрагмент кода запускается только один раз
setup = '''
import numpy as np
import pandas as pd
messy_names = pd.Series(['andrew ', 'bo;b', '  claire  '])
def cleanup(name):
    name = name.replace(';', '')
    name = name.strip()
    name = name.capitalize()
    return name
'''

# Это фрагменты кода, для которых мы будем измерять время выполнения
# (они запускаются много раз)
stmt_pandas_str = '''
messy_names.str.replace(';', '').str.strip().str.capitalize()
'''

stmt_pandas_apply = '''
messy_names.apply(cleanup)
'''

stmt_pandas_vectorize = '''
np.vectorize(cleanup)(messy_names)
'''

In [100]:
timeit.timeit(setup = setup, 
                      stmt = stmt_pandas_str, 
                      number = 10000)

1.0990842999999586

In [102]:
timeit.timeit(setup = setup, 
              stmt = stmt_pandas_apply, 
              number = 10000)

0.20814139999856707

In [106]:
timeit.timeit(setup = setup, 
             stmt = stmt_pandas_vectorize, 
             number = 10000)

0.1606656000003568

Хотя методы __.str()__ могут быть очень удобными, когда дело доходит до повышения производительности, важно помнить про __np.vectorize()__!